In [7]:
import os
import json
import sys
import pandas as pd
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

In [2]:
dirs = ["biorxiv_medrxiv", "comm_use_subset", "noncomm_use_subset"]
articles = []

In [3]:
path = f"kaggle_data/{dirs[0]}/{dirs[0]}/pdf_json"
for file_ in tqdm(os.listdir(path)):
    with open(f"{path}/{file_}") as f:
        data = json.load(f)

        paper_id = data['paper_id']
        title = data['metadata']['title']

        full_abstract = ""
        for part in data['abstract']:
            full_abstract += part['text'] + "\n"

        full_text = ""
        for text_part in data["body_text"]:
            full_text += text_part["text"] + "\n\n"
        
        articles.append((paper_id, title, full_abstract, full_text))

100%|██████████| 1625/1625 [00:01<00:00, 1588.84it/s]


In [4]:
df = pd.DataFrame(articles, columns=['Paper ID', 'Title', 'Abstract', 'Text'])

In [5]:
df.head()

,Paper ID,Title,Abstract,Text
0,7fdd33661d188e3ea250e99f2752d705b5467ef9,Glycoinformatics approach for identifying targ...,COVID-19 outbreak is still threatening the pub...,The coronavirus spike protein (S) plays a key ...
1,ffbd7555a337706238c211197b221795e4e35146,Title: Estimation of COVID-2019 burden and pot...,,CC-BY-NC-ND 4.0 International license It is ma...
2,9a0c89a9b8ff3969d97b7cc4245489ff69f3a615,Development and external validation of a progn...,"Since the outbreak of COVID-19, there has been...",Background: COVID-19 pandemic has developed ra...
3,08911cdc65e71e6398ca79b46806e6c8b2b730ae,Epigenetic dysregulation of ACE2 and interfero...,medRxiv preprint immunology 2015; 156(1): 19-2...,Infection caused by SARS-CoV-2 can result in s...
4,bdaa40d95b82093f60a1c5ac8b798d67cef3a52b,A SARS-CoV-2 Vaccination Strategy Focused on P...,Here we propose a vaccination strategy for SAR...,The current SARS-CoV-2 pandemic has precipitat...


## Find all articles where 'incubator' word was used 

In [13]:
df_incubator = df[df['Text'].str.contains('incubator')]

def how_many_mentioned(text: str, keyword="incubator"):
    counter = 0
    for word in text.split():
        if word == keyword:
            counter += 1
    return counter

df_incubator["Mentioned"] = df_incubator["Text"].progress_apply(lambda text: how_many_mentioned(text))

/home/flush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [19]:
df_incubator

,Paper ID,Title,Abstract,Text,Mentioned
121,63b645b7d8cd2502bedca374d0418486838bf4a0,CORRELATION OF MRNA DELIVERY TIMING AND PROTEI...,Non-viral gene delivery is constrained by the ...,Gene based therapies advance the delivery of e...,1
123,3bdbda7b9bad2d94bfc88950c9efd33c90fb94b6,A SARS-CoV-2-Human Protein-Protein Interaction...,An outbreak of the novel coronavirus SARS-CoV-...,Polymerase (RdRP) inhibitor remdesivir [9] [10...,0
136,295036c0448244642eb6533de571def6cd8437e5,A speed-fidelity trade-off determines the muta...,Mutation rates can evolve through genetic drif...,6 (A6176G) to r-selection over serial passage ...,0
298,b7f94d0b929301cc7a5d8f68a74d4f0270c74649,Electroporated recombinant proteins as tools f...,Delivery of native or chemically modified reco...,"can reach very high concentrations, we observe...",1
360,f9a5880da2dbac60b06c520c9af75985d5d6c24c,The biomechanical role of extra-axonemal struc...,We propose and discuss a model for flagellar m...,Flagella and cilia propel swimming eukaryotic ...,1
363,cdfc8be07677f50c91106e5b56379acb0d0eb289,Stress-induced phospho-ubiquitin formation cau...,Mutations in the E3 ubiquitin ligase parkin ar...,Parkinson's disease (PD) is a debilitating neu...,3
380,ff6bd29c8ecd325c4edddb2b050d5f4919db6112,Deep sequencing of primary human lung epitheli...,15 Current prophylactic and therapeutic strate...,regulates the innate immune response in a clin...,1
469,daf507bcd9337e05d4bdc3da73e05639837f2031,TLR5 participates in the TLR4 receptor complex...,Lung disease causes significant morbidity and ...,Lung disease is a major contributor to morbidi...,1
514,7c8152250ebc8a8b31066de90c3bdaebc076365b,Anaesthetic managment and clinical outcomes of...,Objective: To analyse the clinical features of...,"Since December 2019, a novel coronavirus (SARS...",1
535,26cb6703ca72bf97887abbc29a40b1bd9d7890f4,Hotspots of aberrant enhancer activity in fibr...,"Fibrolamellar carcinoma (FLC) is a rare, thera...",Fibrolamellar carcinoma (FLC) is a rare type o...,1
